# DWH Code

The goal of preparing this jupyter is to source information on the Drought in California. Drought is one of the factors that increase forest fires, so this is an important input factor. 

---
**Project Title:** California wildfires

**Modul:** Data Warehouse and Data Lake Systems 2 (FS 2022)

**Autor:** Maren Forrer

**Browser:** Firefox (Version 99)

**Python:** Python 3.9.5

**Einreichungsdatum:** 09.06.2022

---

In [1]:
from platform import python_version

print(python_version())

3.8.5


In [1]:
# import packages
import psycopg2 # pip install psycopg2-binary
import json # already given
import pandas as pd
import numpy as np
import requests
import datetime
from datetime import date
from datetime import datetime, timedelta



## Connection to DWH (RDS)

In [2]:
# Read credentials from json
credentialsFile = open('credentialsForrer.json')
credentials = json.load(credentialsFile)
credentialsFile.close()
credentialsDic = credentials['credentialsForrer'][0]


In [3]:
# Connect with the RDS - via credentials from json
connectionRequest = "host=" + credentialsDic['rds_host'] + "  dbname=" + credentialsDic['db_name'] + " user=" + credentialsDic['name'] + " password=" + credentialsDic['password']
try: 
    conn = psycopg2.connect(connectionRequest)
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
 
   

In [5]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    # Auto commit is very important
conn.set_session(autocommit=True)

In [6]:
# Only needed if the table is not empty
#cur.execute("TRUNCATE TABLE Drought")

In [8]:
cur.execute("select count(*) from  D_Drought;")
print(cur.fetchall())

[(67570,)]


# Load of the data from Data Lake into the data Warehouse
Since I used the same RDS platform for the Data Lake and the Data Warehouse only the weather data was uploaded in the current RDS environment. 
It was not possible to set up an AWS account with one of our second email adresses, hence we decided to work on with the same environment. 
Further, it would be possible to dublicate the datalake tables and upload them as statging tabels - that would be very easy but since the target is also to work efficient and with low costs. Only the new data will be uploaded as staging tables. 
The following Tables are used as staging tables:
- Firedata_History
- Drought_History
- Staged_historical_weather


# Set up the DWH Infrastructure
Including the helper tables (since each Key needs his own table) 

## Generate the time key 
Since each key needs a seperate tabel two time tables are needed - a weekly and a daily table, with respective keys. 

In [12]:
# generate the date-help table - for this table the date_str is the primary key and week_str the foreign key. 
import pandas as pd
from datetime import datetime
# generate a list with date values over a given period
list_date = pd.date_range(start="2000-01-01",end="2030-01-01").to_pydatetime().tolist()
df_date = pd.DataFrame(list_date)
# add the needed columns to be able to do keys and set up the connections 
df_date.columns = ['date']
df_date['day'] = df_date['date'].dt.strftime('%d')
df_date['week'] = df_date['date'].dt.strftime("%U")
df_date['month'] = df_date['date'].dt.strftime('%m')
df_date['year'] = df_date['date'].dt.strftime('%Y')
df_date['day_name'] = df_date['date'].dt.strftime("%A")
df_date['week_str'] = df_date['date'].dt.strftime("%U-%Y")
df_date['week_str']  = df_date['week_str'] .astype('str')
df_date['week_str']  = df_date['week_str'].str.replace('-','') # will be used as key for the weekly draugth data
df_date['date_str']  = df_date['date'] .astype('str')
df_date['date_str']  = df_date['date_str'].str.replace('-','') # needed for the fire data & to built the Composite_key
df_date.insert(0, 'timeid', range(1, 1 + len(df_date)))
df_date.head(10)

,timeid,date,day,week,month,year,day_name,week_str,date_str
0,1,2000-01-01,01,00,01,2000,Saturday,002000,20000101
1,2,2000-01-02,02,01,01,2000,Sunday,012000,20000102
2,3,2000-01-03,03,01,01,2000,Monday,012000,20000103
3,4,2000-01-04,04,01,01,2000,Tuesday,012000,20000104
4,5,2000-01-05,05,01,01,2000,Wednesday,012000,20000105
5,6,2000-01-06,06,01,01,2000,Thursday,012000,20000106
6,7,2000-01-07,07,01,01,2000,Friday,012000,20000107
7,8,2000-01-08,08,01,01,2000,Saturday,012000,20000108
8,9,2000-01-09,09,02,01,2000,Sunday,022000,20000109
9,10,2000-01-10,10,02,01,2000,Monday,022000,20000110


In [13]:
# weekid -  for this table (Help_Time_Weekly)  the week_str is the primary key and will be used as foreign key in the Help_Time table. 

df_date_week = df_date.copy(deep=True)
df_date_week = df_date_week.drop(columns=['timeid','date','day', 'month', 'day_name', 'date_str'])
df_date_week

#df_date_week_df.weekid.unique()

# get the unique values (rows)
df_date_week_df_2 = df_date_week.drop_duplicates()
df_date_week_df_2 
# drop not needed columns
df_date_week_df_3 = df_date_week_df_2 .drop(columns=['week', 'year'])
# map the weekly key to the complete timeline
df_date_week_df_3 .insert(0, 'weekid', range(1, 1 + len(df_date_week_df_3)))
#df_date_week_id
df_date_week_df_3

,weekid,week_str
0,1,002000
1,2,012000
8,3,022000
15,4,032000
22,5,042000
...,...,...
10935,1589,492029
10942,1590,502029
10949,1591,512029
10956,1592,522029


In [14]:
# add weekid as FK to df_date
df_date = df_date.merge(df_date_week_df_3 , on='week_str' , how ='left')
df_date

,timeid,date,day,week,month,year,day_name,week_str,date_str,weekid
0,1,2000-01-01,01,00,01,2000,Saturday,002000,20000101,1
1,2,2000-01-02,02,01,01,2000,Sunday,012000,20000102,2
2,3,2000-01-03,03,01,01,2000,Monday,012000,20000103,2
3,4,2000-01-04,04,01,01,2000,Tuesday,012000,20000104,2
4,5,2000-01-05,05,01,01,2000,Wednesday,012000,20000105,2
...,...,...,...,...,...,...,...,...,...,...
10954,10955,2029-12-28,28,51,12,2029,Friday,512029,20291228,1591
10955,10956,2029-12-29,29,51,12,2029,Saturday,512029,20291229,1591
10956,10957,2029-12-30,30,52,12,2029,Sunday,522029,20291230,1592
10957,10958,2029-12-31,31,52,12,2029,Monday,522029,20291231,1592


## Generate the Country key 
Since the countries are named differently in the datasets, we need to harmonize the naming. 

In [15]:
# second helper table - counties
# take the values from the stating table
query="SELECT DISTINCT county FROM D_Drought "
cur.execute(query)
county_list = []
for names in cur:
    county_list.append(names)

# generate a df and rename column name
df_county = pd.DataFrame(county_list)
df_county.columns = ['county_dought']
# copy and adjust name - same format as in fire data hist
df_county['county_fire']= df_county['county_dought']
# relace the names to be able to map/connect the same counties together
df_county['county_fire'] = df_county['county_fire'].str.replace(' County', '')
df_county.insert(0, 'countyid', range(1, 1 + len(df_county)))
print(str(len(df_county)) + " the len of the df is equal to the number of counties in California.")
df_county.head(2)

58 the len of the df is equal to the number of counties in California.


,countyid,county_dought,county_fire
0,1,Sacramento County,Sacramento
1,2,Modoc County,Modoc


## Generate the Composite Key
We have need a complex system to connect the entries within the data warehouse, since we have many dependencies. 
1) the correct entries from the different tables have to be connected
- Fire data - daily data with location (longitude and latitude) 
- Draught data - weekly data with country name
- Weather data - for each location (longitude and latitude) of a fire event the daily weather data for 5 fire event are available (5 data points form before 2 days until after 2 days of the fire event). 
2) built a unique key
the key is built from the combination of the date, longitude, and latitude. - all these values are transformed as follows:
- date -> str_date without any special characters

Code below (not the next cell the one after) 

- latitude -> the number without dot or special character (i.e. minus numbers become positive numbers and all decimal places are moved in front of the decimal point and then transformed to an integer)

Code below (not the next cell the one after) 

- longitude -> the number without dot or special character (i.e. minus numbers become positive numbers and all decimal places are moved in front of the decimal point and then transformed to an integer)

Code below (not the next cell the one after) 

Then the single transformed values are connected with an ender and the ID forms the key. This logic can be applied in the Fire and Weather tables and thus these two tables can be connected to each other. 

For the Weather table there is another stumbling block, which will be shown later. 


In [16]:
# additional key between fire weather data and weather data, because currently not linkable. 

# first generate a df with all needed information from D_Fire (since d Fire is the main table, 
# weather data is always loaded for existing fires only).


# Object ID
query="SELECT OBJECTID FROM D_Fire "
cur.execute(query)
pk = []
for names in cur:
    pk.append(names)
pk = pd.DataFrame(pk)
pk.columns = ['OBJECTID']

# first part of key
query="SELECT firediscoverydatetime FROM D_Fire "
#query="SELECT fireoutdatetime FROM D_Fire "
cur.execute(query)
fire_keyWeather_1 = []
for names in cur:
    fire_keyWeather_1.append(names)
fire_keyWeather_1 = pd.DataFrame(fire_keyWeather_1)
fire_keyWeather_1.columns = ['Date']

# second part of key
query="SELECT initiallatitude FROM D_Fire "
cur.execute(query)
fire_keyWeather_2 = []
for names in cur:
    fire_keyWeather_2.append(names)
fire_keyWeather_2 = pd.DataFrame(fire_keyWeather_2)
fire_keyWeather_2.columns = ['Latitude']

# third part of key
query="SELECT initiallongitude FROM D_Fire "
cur.execute(query)
fire_keyWeather_3 = []
for names in cur:
    fire_keyWeather_3.append(names)
fire_keyWeather_3 = pd.DataFrame(fire_keyWeather_3)
fire_keyWeather_3.columns = ['Longitude']

# join the df together

fire_keyWeather = pd.concat([pk, fire_keyWeather_1, fire_keyWeather_2, fire_keyWeather_3], axis=1)
fire_keyWeather

,OBJECTID,Date,Latitude,Longitude
0,139,2016-05-05,38.767220,-119.8167
1,287,2015-07-05,41.421398,-122.4981
2,510,2019-08-10,41.046330,-122.0796
3,716,2017-08-07,41.357780,-120.9694
4,901,2017-08-06,37.751220,-119.1410
...,...,...,...,...
1459,224088,2021-08-30,34.201170,-116.7913
1460,225700,2021-09-10,36.566880,-118.8115
1461,225740,2021-09-10,36.507250,-118.7675
1462,225745,2021-09-10,36.670900,-118.8055


In [17]:
# first generate the primary Composite Key 

# additional key between Fire Weather data and Weather data, as currently there not conecctable. 

# make a numeric key - table Composite_key

fire_wether_key_numeric_table = fire_keyWeather.copy(deep=True)
'''  date -> str_date without any special characters '''
fire_wether_key_numeric_table['num_key_1'] = fire_wether_key_numeric_table['Date']
fire_wether_key_numeric_table['num_key_1']  = fire_wether_key_numeric_table['num_key_1'].astype('str')
fire_wether_key_numeric_table['num_key_1'] = fire_wether_key_numeric_table['num_key_1'].str.replace('-','')

'''  latitude -> the number without dot or special character (i.e. minus numbers 
     become positive numbers and all decimal places are moved in front of the decimal point and then transformed to an integer)'''
fire_wether_key_numeric_table['num_key_2'] = fire_wether_key_numeric_table['Latitude']
fire_wether_key_numeric_table['num_key_2']  = fire_wether_key_numeric_table['num_key_2'].astype('str')
fire_wether_key_numeric_table['num_key_2'] = fire_wether_key_numeric_table['num_key_2'].str.replace('.','')
fire_wether_key_numeric_table['num_key_2'] = fire_wether_key_numeric_table['num_key_2'].str.replace('-','')
 
'''  longitude -> the number without dot or special character (i.e. minus numbers become positive numbers and all decimal 
     places are moved in front of the decimal point and then transformed to an integer) '''
fire_wether_key_numeric_table['num_key_3'] = fire_wether_key_numeric_table['Longitude']
fire_wether_key_numeric_table['num_key_3']  = fire_wether_key_numeric_table['num_key_3'].astype('str')
fire_wether_key_numeric_table['num_key_3'] = fire_wether_key_numeric_table['num_key_3'].str.replace('.','')
fire_wether_key_numeric_table['num_key_3'] = fire_wether_key_numeric_table['num_key_3'].str.replace('-','')

# join the keys into 1 column - generate a unique key:
fire_wether_key_numeric_table_2 = fire_wether_key_numeric_table[['num_key_1', 'num_key_2' ,'num_key_3']].agg(lambda x: ''.join(x.values), axis=1).T

# make a df and rename title
fire_wether_key_numeric_table_2  = pd.DataFrame(fire_wether_key_numeric_table_2)
fire_wether_key_numeric_table_2.columns = ['Key']
fire_wether_key_numeric_table_2 

fire_wether_key_numeric_table_2

# Ass the Key and ID

fire_keyWeather_Key_ID = pd.concat([pk, fire_wether_key_numeric_table_2], axis=1)
fire_keyWeather_Key_ID


,OBJECTID,Key
0,139,2016050538767221198167
1,287,20150705414213981224981
2,510,2019081041046331220796
3,716,2017080741357781209694
4,901,201708063775122119141
...,...,...
1459,224088,2021083034201171167913
1460,225700,2021091036566881188115
1461,225740,2021091036507251187675
1462,225745,202109103667091188055


In [18]:
# generate the fire_key_table - the compositiy key table
fire_key_df = pd.concat([fire_keyWeather, fire_wether_key_numeric_table_2], axis=1)
fire_key_df.insert(0, 'connectid', range(1, 1 + len(fire_key_df)))
fire_key_df

,connectid,OBJECTID,Date,Latitude,Longitude,Key
0,1,139,2016-05-05,38.767220,-119.8167,2016050538767221198167
1,2,287,2015-07-05,41.421398,-122.4981,20150705414213981224981
2,3,510,2019-08-10,41.046330,-122.0796,2019081041046331220796
3,4,716,2017-08-07,41.357780,-120.9694,2017080741357781209694
4,5,901,2017-08-06,37.751220,-119.1410,201708063775122119141
...,...,...,...,...,...,...
1459,1460,224088,2021-08-30,34.201170,-116.7913,2021083034201171167913
1460,1461,225700,2021-09-10,36.566880,-118.8115,2021091036566881188115
1461,1462,225740,2021-09-10,36.507250,-118.7675,2021091036507251187675
1462,1463,225745,2021-09-10,36.670900,-118.8055,202109103667091188055


# Load all Stating tables with the key and into the final Tables
Here to steps are done 1) all tables are created with the interaction keys 2) the Key as set as a Key withing the DWH

## Fire data
Firedata_History form the Data Lake is taken (the transformation was already done in the Data Lake) the keys are added and the table is uploaed to RDS and the keys are set. 

In [19]:
# source complete Table form PostdegreSQL (Data Lake = Staging area DWH) 

query="SELECT * FROM Firedata_History "
cur.execute(query)
Fire = []
for names in cur:
    Fire.append(names)
Fire = pd.DataFrame(Fire)
# add columnames
Fire
Fire.columns = ['OBJECTID', 'ContainmentDateTime', 'ControlDateTime' , 'DailyAcres' , 'DiscoveryAcres', 'FireCause' , 'FireCauseGeneral' , 'FireCauseSpecific' , 'FireDiscoveryDateTime', 'FireMgmtComplexity', 'FireOutDateTime' , 'IncidentName', 'IncidentTypeCategory' , 'IncidentTypeKind' , 'InitialLatitude' , 'InitialLongitude' , 'InitialResponseAcres' , 'POOCity', 'POOCounty' , 'POOLandownerKind', 'POOState' , 'dateOfUpload']
Fire

,OBJECTID,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,FireCause,FireCauseGeneral,FireCauseSpecific,FireDiscoveryDateTime,FireMgmtComplexity,...,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,POOCity,POOCounty,POOLandownerKind,POOState,dateOfUpload
0,139,2016-05-05,2016-05-05,0.10,0.1,Natural,None,None,2016-05-05,None,...,WF,FI,38.767220,-119.8167,0.1,None,Alpine,None,US-CA,2022-04-21
1,287,2015-07-05,2015-07-10,0.20,0.1,Natural,None,None,2015-07-05,None,...,WF,FI,41.421398,-122.4981,NaN,None,Siskiyou,Federal,US-CA,2022-04-21
2,510,2019-08-11,2019-08-11,0.10,0.1,Natural,None,None,2019-08-10,None,...,WF,FI,41.046330,-122.0796,NaN,None,Shasta,Federal,US-CA,2022-04-21
3,716,2017-08-07,2017-08-20,0.10,0.1,Natural,None,None,2017-08-07,None,...,WF,FI,41.357780,-120.9694,0.1,None,Modoc,Federal,US-CA,2022-04-21
4,901,2017-08-06,2017-08-07,0.10,0.1,Natural,None,None,2017-08-06,None,...,WF,FI,37.751220,-119.1410,0.1,None,Mono,Federal,US-CA,2022-04-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,224088,2021-08-31,2021-09-02,0.10,0.1,Natural,Other Natural Cause,None,2021-08-30,None,...,WF,FI,34.201170,-116.7913,NaN,None,San Bernardino,Federal,US-CA,2022-04-21
1460,225700,2021-12-31,2021-12-31,80102.75,2.0,Natural,None,None,2021-09-10,Type 2 Incident,...,WF,FI,36.566880,-118.8115,2.0,None,Tulare,Federal,US-CA,2022-04-21
1461,225740,2021-12-31,2021-12-31,8203.00,0.5,Natural,None,None,2021-09-10,Type 2 Incident,...,WF,FI,36.507250,-118.7675,NaN,Three Rivers,Tulare,Federal,US-CA,2022-04-21
1462,225745,2021-10-14,2021-12-31,1.25,2.0,Natural,None,None,2021-09-10,None,...,WF,FI,36.670900,-118.8055,NaN,None,Tulare,Federal,US-CA,2022-04-21


In [20]:
# add all needed columns to be able to built the key value to join later on the key id
# date:
Fire['date_str'] = Fire['FireDiscoveryDateTime']
Fire['date_str']  = Fire['date_str'].astype('str')
Fire['date_str'] = Fire['date_str'].str.replace('-','')

# county:
Fire['county_fire'] = Fire['POOCounty']

# connect
#Fire['Key'] = fire_wether_key_numeric_table_2 # nicht nötig da über Object ID gemapped werden kann
Fire

,OBJECTID,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,FireCause,FireCauseGeneral,FireCauseSpecific,FireDiscoveryDateTime,FireMgmtComplexity,...,InitialLatitude,InitialLongitude,InitialResponseAcres,POOCity,POOCounty,POOLandownerKind,POOState,dateOfUpload,date_str,county_fire
0,139,2016-05-05,2016-05-05,0.10,0.1,Natural,None,None,2016-05-05,None,...,38.767220,-119.8167,0.1,None,Alpine,None,US-CA,2022-04-21,20160505,Alpine
1,287,2015-07-05,2015-07-10,0.20,0.1,Natural,None,None,2015-07-05,None,...,41.421398,-122.4981,NaN,None,Siskiyou,Federal,US-CA,2022-04-21,20150705,Siskiyou
2,510,2019-08-11,2019-08-11,0.10,0.1,Natural,None,None,2019-08-10,None,...,41.046330,-122.0796,NaN,None,Shasta,Federal,US-CA,2022-04-21,20190810,Shasta
3,716,2017-08-07,2017-08-20,0.10,0.1,Natural,None,None,2017-08-07,None,...,41.357780,-120.9694,0.1,None,Modoc,Federal,US-CA,2022-04-21,20170807,Modoc
4,901,2017-08-06,2017-08-07,0.10,0.1,Natural,None,None,2017-08-06,None,...,37.751220,-119.1410,0.1,None,Mono,Federal,US-CA,2022-04-21,20170806,Mono
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,224088,2021-08-31,2021-09-02,0.10,0.1,Natural,Other Natural Cause,None,2021-08-30,None,...,34.201170,-116.7913,NaN,None,San Bernardino,Federal,US-CA,2022-04-21,20210830,San Bernardino
1460,225700,2021-12-31,2021-12-31,80102.75,2.0,Natural,None,None,2021-09-10,Type 2 Incident,...,36.566880,-118.8115,2.0,None,Tulare,Federal,US-CA,2022-04-21,20210910,Tulare
1461,225740,2021-12-31,2021-12-31,8203.00,0.5,Natural,None,None,2021-09-10,Type 2 Incident,...,36.507250,-118.7675,NaN,Three Rivers,Tulare,Federal,US-CA,2022-04-21,20210910,Tulare
1462,225745,2021-10-14,2021-12-31,1.25,2.0,Natural,None,None,2021-09-10,None,...,36.670900,-118.8055,NaN,None,Tulare,Federal,US-CA,2022-04-21,20210910,Tulare


In [21]:
# add Key_Weather to all rows from all weather data
# key connect
join_Fire_1 = Fire.merge(fire_key_df, on='OBJECTID', how='left')
#join_Fire_1 = Fire.merge(fire_keyWeather_Key_ID, on='OBJECTID', how='left')
# add date
join_Fire_2 = join_Fire_1.merge(df_date , on='date_str' , how ='left')
# add
join_Fire_3 = join_Fire_2.merge(df_county , on='county_fire' , how ='left')
join_Fire_3

,OBJECTID,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,FireCause,FireCauseGeneral,FireCauseSpecific,FireDiscoveryDateTime,FireMgmtComplexity,...,date,day,week,month,year,day_name,week_str,weekid,countyid,county_dought
0,139,2016-05-05,2016-05-05,0.10,0.1,Natural,None,None,2016-05-05,None,...,2016-05-05,05,18,05,2016,Thursday,182016,868,19,Alpine County
1,287,2015-07-05,2015-07-10,0.20,0.1,Natural,None,None,2015-07-05,None,...,2015-07-05,05,27,07,2015,Sunday,272015,824,31,Siskiyou County
2,510,2019-08-11,2019-08-11,0.10,0.1,Natural,None,None,2019-08-10,None,...,2019-08-10,10,31,08,2019,Saturday,312019,1040,48,Shasta County
3,716,2017-08-07,2017-08-20,0.10,0.1,Natural,None,None,2017-08-07,None,...,2017-08-07,07,32,08,2017,Monday,322017,934,2,Modoc County
4,901,2017-08-06,2017-08-07,0.10,0.1,Natural,None,None,2017-08-06,None,...,2017-08-06,06,32,08,2017,Sunday,322017,934,39,Mono County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,224088,2021-08-31,2021-09-02,0.10,0.1,Natural,Other Natural Cause,None,2021-08-30,None,...,2021-08-30,30,35,08,2021,Monday,352021,1150,30,San Bernardino County
1460,225700,2021-12-31,2021-12-31,80102.75,2.0,Natural,None,None,2021-09-10,Type 2 Incident,...,2021-09-10,10,36,09,2021,Friday,362021,1151,12,Tulare County
1461,225740,2021-12-31,2021-12-31,8203.00,0.5,Natural,None,None,2021-09-10,Type 2 Incident,...,2021-09-10,10,36,09,2021,Friday,362021,1151,12,Tulare County
1462,225745,2021-10-14,2021-12-31,1.25,2.0,Natural,None,None,2021-09-10,None,...,2021-09-10,10,36,09,2021,Friday,362021,1151,12,Tulare County


In [22]:
clean_join_Fire_3 = join_Fire_3.copy(deep=True)

In [23]:
clean_join_Fire_3  = clean_join_Fire_3.drop(columns=['date_str' ,'county_fire','Date', 'Key','Latitude' ,'Longitude' , 'county_dought', 'date', 'day' ,'week' ,'month' ,'year' ,'day_name' ] )
clean_join_Fire_3 


,OBJECTID,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,FireCause,FireCauseGeneral,FireCauseSpecific,FireDiscoveryDateTime,FireMgmtComplexity,...,POOCity,POOCounty,POOLandownerKind,POOState,dateOfUpload,connectid,timeid,week_str,weekid,countyid
0,139,2016-05-05,2016-05-05,0.10,0.1,Natural,None,None,2016-05-05,None,...,None,Alpine,None,US-CA,2022-04-21,1,5970,182016,868,19
1,287,2015-07-05,2015-07-10,0.20,0.1,Natural,None,None,2015-07-05,None,...,None,Siskiyou,Federal,US-CA,2022-04-21,2,5665,272015,824,31
2,510,2019-08-11,2019-08-11,0.10,0.1,Natural,None,None,2019-08-10,None,...,None,Shasta,Federal,US-CA,2022-04-21,3,7162,312019,1040,48
3,716,2017-08-07,2017-08-20,0.10,0.1,Natural,None,None,2017-08-07,None,...,None,Modoc,Federal,US-CA,2022-04-21,4,6429,322017,934,2
4,901,2017-08-06,2017-08-07,0.10,0.1,Natural,None,None,2017-08-06,None,...,None,Mono,Federal,US-CA,2022-04-21,5,6428,322017,934,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,224088,2021-08-31,2021-09-02,0.10,0.1,Natural,Other Natural Cause,None,2021-08-30,None,...,None,San Bernardino,Federal,US-CA,2022-04-21,1460,7913,352021,1150,30
1460,225700,2021-12-31,2021-12-31,80102.75,2.0,Natural,None,None,2021-09-10,Type 2 Incident,...,None,Tulare,Federal,US-CA,2022-04-21,1461,7924,362021,1151,12
1461,225740,2021-12-31,2021-12-31,8203.00,0.5,Natural,None,None,2021-09-10,Type 2 Incident,...,Three Rivers,Tulare,Federal,US-CA,2022-04-21,1462,7924,362021,1151,12
1462,225745,2021-10-14,2021-12-31,1.25,2.0,Natural,None,None,2021-09-10,None,...,None,Tulare,Federal,US-CA,2022-04-21,1463,7924,362021,1151,12


In [24]:
# tranform the key incl. primary key to preapre FK in table
fire_keyWeather_Key_ID_2 = fire_keyWeather_Key_ID.copy(deep=True)

fire_keyWeather_Key_ID_2.insert(0, 'FK_Conn', range(1, 1 + len(fire_keyWeather_Key_ID_2)))
fire_keyWeather_Key_ID_2


,FK_Conn,OBJECTID,Key
0,1,139,2016050538767221198167
1,2,287,20150705414213981224981
2,3,510,2019081041046331220796
3,4,716,2017080741357781209694
4,5,901,201708063775122119141
...,...,...,...
1459,1460,224088,2021083034201171167913
1460,1461,225700,2021091036566881188115
1461,1462,225740,2021091036507251187675
1462,1463,225745,202109103667091188055


## Drought data
Drought_History form the Data Lake is taken (the transformation was already done in the Data Lake) the keys are added and the table is uploaed to RDS and the keys are set. 

In [25]:


query="SELECT * FROM  Drought_History"
cur.execute(query)
Drought = []
for names in cur:
    Drought.append(names)
Drought = pd.DataFrame(Drought)
# add columnames
Drought
Drought.columns = [ 'MapDate' ,'FIPS', 'County'  ,'State', 'None'  ,'D0' ,'D1'  ,'D2'  ,'D3' ,'D4' ,'ValidStart' ,'ValidEnd'  ,'StatisticFormatID', 'dateOfUpload']
                
##cur.execute("CREATE TABLE IF NOT EXISTS Drought_History ( MapDate varchar(10) ,FIPS varchar(10), County varchar(255) ,State varchar(10), None FLOAT ,D0 FLOAT ,D1 FLOAT ,D2 FLOAT ,D3 FLOAT ,D4 FLOAT,ValidStart DATE,ValidEnd DATE ,StatisticFormatID INTEGER, dateOfUpload DATE)")

Drought

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,dateOfUpload
0,20220322,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-22,2022-03-28,1,2022-04-21
1,20220315,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-15,2022-03-21,1,2022-04-21
2,20220308,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-08,2022-03-14,1,2022-04-21
3,20220301,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-01,2022-03-07,1,2022-04-21
4,20220222,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-02-22,2022-02-28,1,2022-04-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67565,20220419,06115,Yuba County,CA,0.0,100.0,100.0,100.0,77.61,0.0,2022-04-19,2022-04-25,1,2022-04-23
67566,20220412,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,2022-04-12,2022-04-18,1,2022-04-23
67567,20220405,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,2022-04-05,2022-04-11,1,2022-04-23
67568,20220329,06115,Yuba County,CA,0.0,100.0,100.0,100.0,38.71,0.0,2022-03-29,2022-04-04,1,2022-04-23


In [26]:
# add droughtid, add timeid and countyid

# add all needed columns
# date:
Drought['date_str'] = Drought['ValidStart']
Drought['date_str']  = Drought['date_str'].astype('str')
Drought['date_str'] = Drought['date_str'].str.replace('-','')

# county:
Drought['county_dought'] = Drought['County']

# connect
#Fire['Key'] = fire_wether_key_numeric_table_2 # nicht nötig da über Object ID gemapped werden kann
Drought

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,dateOfUpload,date_str,county_dought
0,20220322,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-22,2022-03-28,1,2022-04-21,20220322,Alameda County
1,20220315,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-15,2022-03-21,1,2022-04-21,20220315,Alameda County
2,20220308,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-08,2022-03-14,1,2022-04-21,20220308,Alameda County
3,20220301,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-01,2022-03-07,1,2022-04-21,20220301,Alameda County
4,20220222,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-02-22,2022-02-28,1,2022-04-21,20220222,Alameda County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67565,20220419,06115,Yuba County,CA,0.0,100.0,100.0,100.0,77.61,0.0,2022-04-19,2022-04-25,1,2022-04-23,20220419,Yuba County
67566,20220412,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,2022-04-12,2022-04-18,1,2022-04-23,20220412,Yuba County
67567,20220405,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,2022-04-05,2022-04-11,1,2022-04-23,20220405,Yuba County
67568,20220329,06115,Yuba County,CA,0.0,100.0,100.0,100.0,38.71,0.0,2022-03-29,2022-04-04,1,2022-04-23,20220329,Yuba County


In [27]:
# add Key_Weather to all rows from all weather data

# add date
join_Drought_1 = Drought.merge(df_date , on='date_str' , how ='left')
# add
join_Drought_2 = join_Drought_1.merge(df_county , on='county_dought' , how ='left')
join_Drought_2

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,...,date,day,week,month,year,day_name,week_str,weekid,countyid,county_fire
0,20220322,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,...,2022-03-22,22,12,03,2022,Tuesday,122022,1180,47,Alameda
1,20220315,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,...,2022-03-15,15,11,03,2022,Tuesday,112022,1179,47,Alameda
2,20220308,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,...,2022-03-08,08,10,03,2022,Tuesday,102022,1178,47,Alameda
3,20220301,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,...,2022-03-01,01,09,03,2022,Tuesday,092022,1177,47,Alameda
4,20220222,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,...,2022-02-22,22,08,02,2022,Tuesday,082022,1176,47,Alameda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67565,20220419,06115,Yuba County,CA,0.0,100.0,100.0,100.0,77.61,0.0,...,2022-04-19,19,16,04,2022,Tuesday,162022,1184,40,Yuba
67566,20220412,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,...,2022-04-12,12,15,04,2022,Tuesday,152022,1183,40,Yuba
67567,20220405,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,...,2022-04-05,05,14,04,2022,Tuesday,142022,1182,40,Yuba
67568,20220329,06115,Yuba County,CA,0.0,100.0,100.0,100.0,38.71,0.0,...,2022-03-29,29,13,03,2022,Tuesday,132022,1181,40,Yuba


In [28]:
clean_join_Drought_2  = join_Drought_2.copy(deep=True)

In [29]:
clean_join_Drought_2 = clean_join_Drought_2.drop(columns=['timeid','county_fire', 'date', 'day' ,'week' ,'month' ,'year' ,'day_name' ,'date_str' ,'county_dought'] )
clean_join_Drought_2



,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,dateOfUpload,week_str,weekid,countyid
0,20220322,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-22,2022-03-28,1,2022-04-21,122022,1180,47
1,20220315,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-15,2022-03-21,1,2022-04-21,112022,1179,47
2,20220308,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-08,2022-03-14,1,2022-04-21,102022,1178,47
3,20220301,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-03-01,2022-03-07,1,2022-04-21,092022,1177,47
4,20220222,06001,Alameda County,CA,0.0,100.0,100.0,100.0,0.00,0.0,2022-02-22,2022-02-28,1,2022-04-21,082022,1176,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67565,20220419,06115,Yuba County,CA,0.0,100.0,100.0,100.0,77.61,0.0,2022-04-19,2022-04-25,1,2022-04-23,162022,1184,40
67566,20220412,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,2022-04-12,2022-04-18,1,2022-04-23,152022,1183,40
67567,20220405,06115,Yuba County,CA,0.0,100.0,100.0,100.0,58.83,0.0,2022-04-05,2022-04-11,1,2022-04-23,142022,1182,40
67568,20220329,06115,Yuba County,CA,0.0,100.0,100.0,100.0,38.71,0.0,2022-03-29,2022-04-04,1,2022-04-23,132022,1181,40


## Weather data
Historical_Weather was taken and the ETL proces was performed (by Nathan) he uploaded the table: Staged_Historical_Weather into my RDS (DWH). Now the keys are added and the table is uploaed to RDS and the keys are set. 

In [30]:

query="SELECT * FROM  Staged_historical_weather"
cur.execute(query)
Weather = []
for names in cur:
    Weather.append(names)
Weather = pd.DataFrame(Weather)
# add columnames
Weather
#Weather.columns = [ 'MapDate' ,'FIPS', 'County'  ,'State', 'None'  ,'D0' ,'D1'  ,'D2'  ,'D3' ,'D4' ,'ValidStart' ,'ValidEnd'  ,'StatisticFormatID', 'dateOfUpload']
Weather.columns = ['Group', 'date', 'latitude', 'longitude', 'tz', 'maxt', 'temp', 'mint', 'wdir', 'spd','wgust','windchill', 'sealevelpressure', 'heatindex', 'solarenergy','solarradiation', 'humidity', 'visibility', 'cloudcover', 'precip', 'precipcover', 'weathertype', 'conditions']
              
##cur.execute("CREATE TABLE IF NOT EXISTS Drought_History ( MapDate varchar(10) ,FIPS varchar(10), County varchar(255) ,State varchar(10), None FLOAT ,D0 FLOAT ,D1 FLOAT ,D2 FLOAT ,D3 FLOAT ,D4 FLOAT,ValidStart DATE,ValidEnd DATE ,StatisticFormatID INTEGER, dateOfUpload DATE)")

Weather

,Group,date,latitude,longitude,tz,maxt,temp,mint,wdir,spd,...,heatindex,solarenergy,solarradiation,humidity,visibility,cloudcover,precip,precipcover,weathertype,conditions
0,id_1,2016-05-03,38.76722,-119.8167,America/Los_Angeles,17.6,9.4,0.3,176.79,11.9,...,NaN,NaN,NaN,64.00,15.9,30.0,0.00,0.00,Light Rain,Partially cloudy
1,id_1,2016-05-04,38.76722,-119.8167,America/Los_Angeles,17.7,10.4,3.4,176.08,18.3,...,NaN,NaN,NaN,62.93,16.0,23.6,0.24,8.33,"Lightning Without Thunder, Light Rain",Snow
2,id_1,2016-05-05,38.76722,-119.8167,America/Los_Angeles,11.5,7.2,2.9,148.46,15.7,...,NaN,NaN,NaN,80.99,14.3,63.4,12.80,54.17,"Lightning Without Thunder, Mist, Light Snow, T...","Snow, Partially cloudy"
3,id_1,2016-05-06,38.76722,-119.8167,America/Los_Angeles,12.9,6.6,2.0,220.33,12.2,...,NaN,NaN,NaN,78.93,13.8,68.7,9.14,58.33,"Mist, Light Snow, Rain, Thunderstorm, Light Rain","Rain, Partially cloudy"
4,id_1,2016-05-07,38.76722,-119.8167,America/Los_Angeles,9.4,7.0,4.9,181.25,10.7,...,NaN,NaN,NaN,87.57,14.8,81.0,4.33,45.83,"Mist, Light Rain","Snow, Overcast"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,id_193,2015-06-09,41.11080,-122.2019,America/Los_Angeles,33.6,23.7,15.9,164.81,13.0,...,32.3,NaN,NaN,53.92,16.0,26.3,0.78,12.50,"Lightning Without Thunder, Light Rain","Rain, Partially cloudy"
961,id_193,2015-06-10,41.11080,-122.2019,America/Los_Angeles,29.8,22.1,15.0,187.17,14.6,...,28.2,NaN,NaN,53.12,16.0,0.1,0.00,0.00,,Clear
962,id_193,2015-06-11,41.11080,-122.2019,America/Los_Angeles,31.6,22.6,12.0,231.70,15.0,...,29.5,NaN,NaN,32.25,16.0,0.0,0.00,0.00,,Clear
963,id_193,2015-06-12,41.11080,-122.2019,America/Los_Angeles,33.9,25.6,17.9,284.25,15.5,...,31.5,NaN,NaN,32.08,16.0,0.0,0.00,0.00,,Clear


In [41]:
'''add connectid and for each day a number [-2,-1,0,+1,+2]
for each fire event 5 data points are sourced - currently it is not availabe which is the fire event date. 
Therefore a logic is implemnted/added. ''' 
s = [-2,-1,0,+1,+2]
firedate = 193*s
Weather['firedate']=firedate

In [38]:
# add needed columns: date_str

Weather['date_str'] = Weather['date']
Weather['date_str']  = Weather['date_str'].astype('str')
Weather['date_str'] = Weather['date_str'].str.replace('-','')
Weather.head(3)

,Group,date,latitude,longitude,tz,maxt,temp,mint,wdir,spd,...,solarradiation,humidity,visibility,cloudcover,precip,precipcover,weathertype,conditions,firedate,date_str
0,id_1,2016-05-03,38.76722,-119.8167,America/Los_Angeles,17.6,9.4,0.3,176.79,11.9,...,NaN,64.00,15.9,30.0,0.00,0.00,Light Rain,Partially cloudy,-2,20160503
1,id_1,2016-05-04,38.76722,-119.8167,America/Los_Angeles,17.7,10.4,3.4,176.08,18.3,...,NaN,62.93,16.0,23.6,0.24,8.33,"Lightning Without Thunder, Light Rain",Snow,-1,20160504
2,id_1,2016-05-05,38.76722,-119.8167,America/Los_Angeles,11.5,7.2,2.9,148.46,15.7,...,NaN,80.99,14.3,63.4,12.80,54.17,"Lightning Without Thunder, Mist, Light Snow, T...","Snow, Partially cloudy",0,20160505


In [44]:
# get the data for the Weather data - hiere we have sourced the weather of 2 days before - the weather of the day of
# fire outbreak and the weather of 2 days after the fireoutbreak

# Weather keys
query="SELECT id FROM D_Weather "
cur.execute(query)
pk = []
for names in cur:
    pk.append(names)
pk = pd.DataFrame(pk)
pk.columns = ['Group']


# first part of key
query="SELECT date FROM D_Weather "
cur.execute(query)
fire_keyWeather_1_2 = []
for names in cur:
    fire_keyWeather_1_2.append(names)
fire_keyWeather_1_2 = pd.DataFrame(fire_keyWeather_1_2)
fire_keyWeather_1_2.columns = ['Date']


# second part of key
query="SELECT Latitude FROM D_Weather "
cur.execute(query)
fire_keyWeather_2_2 = []
for names in cur:
    fire_keyWeather_2_2.append(names)
fire_keyWeather_2_2 = pd.DataFrame(fire_keyWeather_2_2)
fire_keyWeather_2_2.columns = ['Latitude']


# 3. part of key
query="SELECT longitude FROM D_Weather "
cur.execute(query)
fire_keyWeather_3_2 = []
for names in cur:
    fire_keyWeather_3_2.append(names)
fire_keyWeather_3_2 = pd.DataFrame(fire_keyWeather_3_2)
fire_keyWeather_3_2.columns = ['Longitude']

In [46]:
# generate a list to replace the Keys - always for 5 weather data the same KEy 
# Overview
fire_keyWeather_1_2['Date']  = fire_keyWeather_1_2['Date'].astype('str')
fire_keyWeather_2_2['Latitude']  = fire_keyWeather_2_2['Latitude'].astype('str')
fire_keyWeather_3_2['Longitude']  = fire_keyWeather_3_2['Longitude'].astype('str')


# make a numeric key 
fire_wether_key_numeric = fire_wether_key_df_4.copy(deep=True)
# replace date
fire_wether_key_numeric['num_key_1'] = fire_wether_key_numeric['Date']
fire_wether_key_numeric['num_key_1']  = fire_wether_key_numeric['num_key_1'].astype('str')
fire_wether_key_numeric['num_key_1'] = fire_wether_key_numeric['num_key_1'].str.replace('-','')

# replace latitude
fire_wether_key_numeric['num_key_2'] = fire_wether_key_numeric['Latitude']
fire_wether_key_numeric['num_key_2']  = fire_wether_key_numeric['num_key_2'].astype('str')
fire_wether_key_numeric['num_key_2'] = fire_wether_key_numeric['num_key_2'].str.replace('.','')
fire_wether_key_numeric['num_key_2'] = fire_wether_key_numeric['num_key_2'].str.replace('-','')

# replace longitude
fire_wether_key_numeric['num_key_3'] = fire_wether_key_numeric['Longitude']
fire_wether_key_numeric['num_key_3']  = fire_wether_key_numeric['num_key_3'].astype('str')
fire_wether_key_numeric['num_key_3'] = fire_wether_key_numeric['num_key_3'].str.replace('.','')
fire_wether_key_numeric['num_key_3'] = fire_wether_key_numeric['num_key_3'].str.replace('-','')

fire_wether_key_numeric

# join the keys into 1:

# generate 1 key
fire_wether_key_numeric_2 = fire_wether_key_numeric[['num_key_1', 'num_key_2' ,'num_key_3']].agg(lambda x: ''.join(x.values), axis=1).T

# make a df and rename title
fire_wether_key_numeric_2  = pd.DataFrame(fire_wether_key_numeric_2)
fire_wether_key_numeric_2.columns = ['Key']
fire_wether_key_numeric_2 

fire_wether_key_numeric_2 # df with all weather keys
#join_weather = join_Fire_1.merge(df_date_2 , on='FireDiscoveryDateTime_2' , how ='left')

,Key
0,2016050338767221198167
1,2016050438767221198167
2,2016050538767221198167
3,2016050638767221198167
4,2016050738767221198167
...,...
960,201506094111081222019
961,201506104111081222019
962,201506114111081222019
963,201506124111081222019


In [47]:


#generate 1 df
fire_wether_key_df_4 = pd.concat([pk, fire_keyWeather_1_2, fire_keyWeather_2_2, fire_keyWeather_3_2], axis=1)
fire_wether_key_df_4


# add the key which was built before
Weather_2 = pd.concat([Weather,fire_wether_key_numeric_2 ], axis=1)
# Key Generating
Weather_Key_Workaround = Weather_2[['Group', 'firedate','Key']]
Weather_Key_Workaround

#Weather_Key_Workaround["Key"]=Weather_Key_Workaround.groupby("Group")["firedate"==0]
#Weather_Key_Workaround["Key"]=Weather_Key_Workaround["Key"].map(Weather_Key_Workaround.groupby("Key")["Group"])
# delet wrong KEy Rows
Weather_Key_Workaround.drop(Weather_Key_Workaround.index[Weather_Key_Workaround['firedate'] != 0 ], inplace=True)
Weather_Key_Workaround

List_Workaround = Weather_Key_Workaround[['Key']]
# loop to generate the right key: 

List_Workaround 
    

C:\Users\maren\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Key
2,2016050538767221198167
7,20150705414213981224981
12,2019081041046331220796
17,2017080741357781209694
22,201708063775122119141
...,...
942,201508293660151188104
947,2017091239603271201812
952,2019080841450341202425
957,2018072138140441187997


In [48]:
List_Workaround = Weather_Key_Workaround[['Key']]
List_Workaround 

,Key
2,2016050538767221198167
7,20150705414213981224981
12,2019081041046331220796
17,2017080741357781209694
22,201708063775122119141
...,...
942,201508293660151188104
947,2017091239603271201812
952,2019080841450341202425
957,2018072138140441187997


In [49]:
count = 0
a=[]

for row in List_Workaround.iterrows():

    line = List_Workaround.iloc[count, 0] 

      #print(line) 
    a.append(line)
    a.append(line)
    a.append(line)
    a.append(line)
    a.append(line)
    count = count+1
    
#a
right_key = pd.DataFrame(a)
right_key.columns = ['Key']
right_key

,Key
0,2016050538767221198167
1,2016050538767221198167
2,2016050538767221198167
3,2016050538767221198167
4,2016050538767221198167
...,...
960,201506114111081222019
961,201506114111081222019
962,201506114111081222019
963,201506114111081222019


In [50]:
# concat the Key and the Weather data 
Weather = pd.concat([Weather,right_key ], axis=1)

In [51]:
Weather.head(2)

,Group,date,latitude,longitude,tz,maxt,temp,mint,wdir,spd,...,humidity,visibility,cloudcover,precip,precipcover,weathertype,conditions,firedate,date_str,Key
0,id_1,2016-05-03,38.76722,-119.8167,America/Los_Angeles,17.6,9.4,0.3,176.79,11.9,...,64.00,15.9,30.0,0.00,0.00,Light Rain,Partially cloudy,-2,20160503,2016050538767221198167
1,id_1,2016-05-04,38.76722,-119.8167,America/Los_Angeles,17.7,10.4,3.4,176.08,18.3,...,62.93,16.0,23.6,0.24,8.33,"Lightning Without Thunder, Light Rain",Snow,-1,20160504,2016050538767221198167


In [52]:
# add Key -> connectid 

join_Weather_1 = Weather.merge(fire_key_df, on='Key', how='left')
join_Weather_1

,Group,date,latitude,longitude,tz,maxt,temp,mint,wdir,spd,...,weathertype,conditions,firedate,date_str,Key,connectid,OBJECTID,Date,Latitude,Longitude
0,id_1,2016-05-03,38.76722,-119.8167,America/Los_Angeles,17.6,9.4,0.3,176.79,11.9,...,Light Rain,Partially cloudy,-2,20160503,2016050538767221198167,1.0,139.0,2016-05-05,38.76722,-119.8167
1,id_1,2016-05-04,38.76722,-119.8167,America/Los_Angeles,17.7,10.4,3.4,176.08,18.3,...,"Lightning Without Thunder, Light Rain",Snow,-1,20160504,2016050538767221198167,1.0,139.0,2016-05-05,38.76722,-119.8167
2,id_1,2016-05-05,38.76722,-119.8167,America/Los_Angeles,11.5,7.2,2.9,148.46,15.7,...,"Lightning Without Thunder, Mist, Light Snow, T...","Snow, Partially cloudy",0,20160505,2016050538767221198167,1.0,139.0,2016-05-05,38.76722,-119.8167
3,id_1,2016-05-06,38.76722,-119.8167,America/Los_Angeles,12.9,6.6,2.0,220.33,12.2,...,"Mist, Light Snow, Rain, Thunderstorm, Light Rain","Rain, Partially cloudy",1,20160506,2016050538767221198167,1.0,139.0,2016-05-05,38.76722,-119.8167
4,id_1,2016-05-07,38.76722,-119.8167,America/Los_Angeles,9.4,7.0,4.9,181.25,10.7,...,"Mist, Light Rain","Snow, Overcast",2,20160507,2016050538767221198167,1.0,139.0,2016-05-05,38.76722,-119.8167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,id_193,2015-06-09,41.11080,-122.2019,America/Los_Angeles,33.6,23.7,15.9,164.81,13.0,...,"Lightning Without Thunder, Light Rain","Rain, Partially cloudy",-2,20150609,201506114111081222019,205.0,28799.0,2015-06-11,41.11080,-122.2019
961,id_193,2015-06-10,41.11080,-122.2019,America/Los_Angeles,29.8,22.1,15.0,187.17,14.6,...,,Clear,-1,20150610,201506114111081222019,205.0,28799.0,2015-06-11,41.11080,-122.2019
962,id_193,2015-06-11,41.11080,-122.2019,America/Los_Angeles,31.6,22.6,12.0,231.70,15.0,...,,Clear,0,20150611,201506114111081222019,205.0,28799.0,2015-06-11,41.11080,-122.2019
963,id_193,2015-06-12,41.11080,-122.2019,America/Los_Angeles,33.9,25.6,17.9,284.25,15.5,...,,Clear,1,20150612,201506114111081222019,205.0,28799.0,2015-06-11,41.11080,-122.2019


In [53]:
# add date informations
join_Weather_2 = join_Weather_1.merge(df_date , on='date_str' , how ='left')
join_Weather_2


,Group,date_x,latitude,longitude,tz,maxt,temp,mint,wdir,spd,...,Longitude,timeid,date_y,day,week,month,year,day_name,week_str,weekid
0,id_1,2016-05-03,38.76722,-119.8167,America/Los_Angeles,17.6,9.4,0.3,176.79,11.9,...,-119.8167,5968,2016-05-03,03,18,05,2016,Tuesday,182016,868
1,id_1,2016-05-04,38.76722,-119.8167,America/Los_Angeles,17.7,10.4,3.4,176.08,18.3,...,-119.8167,5969,2016-05-04,04,18,05,2016,Wednesday,182016,868
2,id_1,2016-05-05,38.76722,-119.8167,America/Los_Angeles,11.5,7.2,2.9,148.46,15.7,...,-119.8167,5970,2016-05-05,05,18,05,2016,Thursday,182016,868
3,id_1,2016-05-06,38.76722,-119.8167,America/Los_Angeles,12.9,6.6,2.0,220.33,12.2,...,-119.8167,5971,2016-05-06,06,18,05,2016,Friday,182016,868
4,id_1,2016-05-07,38.76722,-119.8167,America/Los_Angeles,9.4,7.0,4.9,181.25,10.7,...,-119.8167,5972,2016-05-07,07,18,05,2016,Saturday,182016,868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,id_193,2015-06-09,41.11080,-122.2019,America/Los_Angeles,33.6,23.7,15.9,164.81,13.0,...,-122.2019,5639,2015-06-09,09,23,06,2015,Tuesday,232015,820
961,id_193,2015-06-10,41.11080,-122.2019,America/Los_Angeles,29.8,22.1,15.0,187.17,14.6,...,-122.2019,5640,2015-06-10,10,23,06,2015,Wednesday,232015,820
962,id_193,2015-06-11,41.11080,-122.2019,America/Los_Angeles,31.6,22.6,12.0,231.70,15.0,...,-122.2019,5641,2015-06-11,11,23,06,2015,Thursday,232015,820
963,id_193,2015-06-12,41.11080,-122.2019,America/Los_Angeles,33.9,25.6,17.9,284.25,15.5,...,-122.2019,5642,2015-06-12,12,23,06,2015,Friday,232015,820


In [110]:
# generate a clean table only with keys
clean_join_Weather_2 = join_Weather_2.copy(deep=True)     


In [111]:
clean_join_Weather_2 = clean_join_Weather_2.drop(columns=['week_str','timeid','date_str', 'Key', 'Latitude' ,'Longitude','OBJECTID' ,'Date' ,'date_y' ,'day' ,'week' ,'month','year', 'day_name' ])
clean_join_Weather_2 = clean_join_Weather_2[clean_join_Weather_2['connectid'].notna()]
clean_join_Weather_2


,Group,date_x,latitude,longitude,tz,maxt,temp,mint,wdir,spd,...,humidity,visibility,cloudcover,precip,precipcover,weathertype,conditions,firedate,connectid,weekid
0,id_1,2016-05-03,38.76722,-119.8167,America/Los_Angeles,17.6,9.4,0.3,176.79,11.9,...,64.00,15.9,30.0,0.00,0.00,Light Rain,Partially cloudy,-2,1.0,868
1,id_1,2016-05-04,38.76722,-119.8167,America/Los_Angeles,17.7,10.4,3.4,176.08,18.3,...,62.93,16.0,23.6,0.24,8.33,"Lightning Without Thunder, Light Rain",Snow,-1,1.0,868
2,id_1,2016-05-05,38.76722,-119.8167,America/Los_Angeles,11.5,7.2,2.9,148.46,15.7,...,80.99,14.3,63.4,12.80,54.17,"Lightning Without Thunder, Mist, Light Snow, T...","Snow, Partially cloudy",0,1.0,868
3,id_1,2016-05-06,38.76722,-119.8167,America/Los_Angeles,12.9,6.6,2.0,220.33,12.2,...,78.93,13.8,68.7,9.14,58.33,"Mist, Light Snow, Rain, Thunderstorm, Light Rain","Rain, Partially cloudy",1,1.0,868
4,id_1,2016-05-07,38.76722,-119.8167,America/Los_Angeles,9.4,7.0,4.9,181.25,10.7,...,87.57,14.8,81.0,4.33,45.83,"Mist, Light Rain","Snow, Overcast",2,1.0,868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,id_193,2015-06-09,41.11080,-122.2019,America/Los_Angeles,33.6,23.7,15.9,164.81,13.0,...,53.92,16.0,26.3,0.78,12.50,"Lightning Without Thunder, Light Rain","Rain, Partially cloudy",-2,205.0,820
961,id_193,2015-06-10,41.11080,-122.2019,America/Los_Angeles,29.8,22.1,15.0,187.17,14.6,...,53.12,16.0,0.1,0.00,0.00,,Clear,-1,205.0,820
962,id_193,2015-06-11,41.11080,-122.2019,America/Los_Angeles,31.6,22.6,12.0,231.70,15.0,...,32.25,16.0,0.0,0.00,0.00,,Clear,0,205.0,820
963,id_193,2015-06-12,41.11080,-122.2019,America/Los_Angeles,33.9,25.6,17.9,284.25,15.5,...,32.08,16.0,0.0,0.00,0.00,,Clear,1,205.0,820


## Upload the tables to RDS (DWH)
- Fire
- Drought
- Weather
- and all tables which are needed because of the keys 

### Upload F_Fire_History

In [57]:
# FIRE
print(len(clean_join_Fire_3))
dataTypeSeries = clean_join_Fire_3.dtypes


1464


In [58]:

cur.execute("CREATE TABLE IF NOT EXISTS F_Fire_History_2 ( OBJECTID integer,	ContainmentDateTime varchar(255),	ControlDateTime varchar(255),	DailyAcres float,	DiscoveryAcres float,	FireCause varchar(255),	FireCauseGeneral varchar(255),	FireCauseSpecific varchar(255),	FireDiscoveryDateTime varchar(255),	FireMgmtComplexity varchar(255),	FireOutDateTime varchar(255),	IncidentName varchar(255),	IncidentTypeCategory varchar(255),	IncidentTypeKind varchar(255),	InitialLatitude float,	InitialLongitude float,	InitialResponseAcres float,	POOCity varchar(255),	POOCounty varchar(255),	POOLandownerKind varchar(255),	POOState varchar(255),	dateOfUpload varchar(255),	connectid integer,	timeid integer,	week_str varchar(255),	weekid integer,	countyid integer )")

In [59]:
# Upload entries 

# rename the DF - in this case the below function must not be changed
data = clean_join_Fire_3

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  F_Fire_History_2 (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()

In [ ]:
# add primary key in pg ADMIN: 
#ALTER TABLE F_Fire_History_2 ADD COLUMN FireID SERIAL PRIMARY KEY;

In [ ]:
#ALTER TABLE distributors ADD CONSTRAINT distfk FOREIGN KEY (address) REFERENCES addresses (address) MATCH FULL;


In [ ]:
# Add FK: county_id
#ALTER TABLE F_Fire_History_2
#ADD CONSTRAINT county_id
#FOREIGN KEY (countyid) 
#REFERENCES Help_County_2  (countyid);

In [ ]:
# Add FK: time_id 
#ALTER TABLE F_Fire_History_2
#ADD CONSTRAINT time_id
#FOREIGN KEY (timeid) 
#REFERENCES H_Date_History_2  (timeid);

In [ ]:
# Add FK: week_id 
#ALTER TABLE F_Fire_History_2
#ADD CONSTRAINT week_id
#FOREIGN KEY (weekid) 
#REFERENCES Help_Time_Weekly_2  (weekid);

In [ ]:
# Add FK: connect_id 
#ALTER TABLE F_Fire_History_2
#ADD CONSTRAINT connect_id 
#FOREIGN KEY (connectid) 
#REFERENCES help_connect_2 (connectid);

### Upload F_Drought_History 

In [60]:
# Drought
print(len(clean_join_Drought_2))
dataTypeSeries = clean_join_Drought_2.dtypes


67570


In [61]:

cur.execute("CREATE TABLE IF NOT EXISTS F_Drought_History_2 (  MapDate  varchar(255),	FIPS  varchar(255),	County  varchar(255),	State  varchar(255),	None float,	D0 float,	D1 float,	D2 float,	D3 float,	D4 float,	ValidStart  varchar(255),	ValidEnd  varchar(255),	StatisticFormatID integer,	dateOfUpload  varchar(255),	week_str  varchar(255),	weekid integer,	countyid integer)")

In [62]:
# Upload entries 

# rename the DF - in this case the below function must not be changed
data = clean_join_Drought_2

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  F_Drought_History_2 (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()

In [ ]:
# add primary key in pg ADMIN: 
#ALTER TABLE F_Drought_History_2 ADD COLUMN droughtid SERIAL PRIMARY KEY;

In [ ]:
## Add the FKs
#-- Add FK: county_id
#ALTER TABLE F_Drought_History_2
#ADD CONSTRAINT county_id
#FOREIGN KEY (countyid) 
#REFERENCES Help_County_2  (countyid);

#--  Add FK: week_id 
##ALTER TABLE F_Drought_History_2
#ADD CONSTRAINT week_id
#FOREIGN KEY (weekid) 
#REFERENCES Help_Time_Weekly_2  (weekid);


### Upload F_Weather_History

In [117]:
# Weather
print(len(clean_join_Weather_2))
dataTypeSeries = clean_join_Weather_2.dtypes
# since Group is not accepted as Column Title - I reset the name. 
clean_join_Weather_2 = clean_join_Weather_2.rename(columns={"Group": "Group_id"})


905


In [119]:

cur.execute("CREATE TABLE IF NOT EXISTS F_Weather_History_2 ( Group_id  varchar(255),	date_x  varchar(255),	latitude float,	longitude float,	tz  varchar(255),	maxt float,	temp float,	mint float,	wdir float,	spd float,	wgust float,	windchill float,	sealevelpressure float,	heatindex float,	solarenergy float,	solarradiation float,	humidity float,	visibility float,	cloudcover float,	precip float,	precipcover float,	weathertype  varchar(255),	conditions  varchar(255),	firedate integer,	connectid integer,	weekid integer )")

In [120]:
# Upload entries 

# rename the DF - in this case the below function must not be changed
data = clean_join_Weather_2

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  F_Weather_History_2 (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()

In [ ]:
# add primary key in pg ADMIN: 
#ALTER TABLE F_Weather_History_2 ADD COLUMN weatherid SERIAL PRIMARY KEY;

In [ ]:
## Add the FKs

#--  Add FK: week_id 
#ALTER TABLE F_Weather_History_2 
#ADD CONSTRAINT week_id
#FOREIGN KEY (weekid) 
#REFERENCES Help_Time_Weekly_2  (weekid);

#--Add FK: connect_id 
#ALTER TABLE F_Weather_History_2 
#ADD CONSTRAINT connect_id 
#FOREIGN KEY (connectid) 
#REFERENCES help_connect_2 (connectid);

### Upload H_Date_History

In [ ]:
# Only needed if the table is not empty
#cur.execute("TRUNCATE TABLE H_Time")

In [ ]:
### Upload H_Date_History

In [66]:
# Date
print(len(df_date))
dataTypeSeries = df_date.dtypes

10959


In [67]:
cur.execute("CREATE TABLE IF NOT EXISTS H_Date_History_2 ( timeid integer,	date date,	day varchar(255),	week varchar(255),	month varchar(255),	year varchar(255),	day_name varchar(255),	week_str varchar(255),	date_str varchar(255),	weekid integer)")

In [68]:
# Upload entries 

# rename the DF - in this case the below function must not be changed
data = df_date

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  H_Date_History_2 (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()

In [ ]:
# add primary key in pg ADMIN: 
#ALTER TABLE IF EXISTS public.h_date_history_2
#    ADD CONSTRAINT timeid PRIMARY KEY (timeid);

In [ ]:
## Add the FKs

#--  Add FK: week_id 
#ALTER TABLE h_date_history_2
#ADD CONSTRAINT week_id
#FOREIGN KEY (weekid) 
#REFERENCES Help_Time_Weekly_2  (weekid);

### Upload H_Date_Weekly_History

In [69]:
# Date_Weekly
print(len(df_date_week_df_3))
dataTypeSeries = df_date_week_df_3.dtypes

1593


In [70]:
# inserat help_time_weekly table to PostdegreSQL: 

cur.execute("CREATE TABLE IF NOT EXISTS Help_Time_Weekly_2 ( weekid INTEGER, week_str INTEGER )")


In [71]:
# upload the entries 

# rename the DF - in this case the below function must not be changed
data = df_date_week_df_3

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  help_time_weekly_2 (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()
    
    


In [ ]:
# add primary key in pg ADMIN: 
#ALTER TABLE IF EXISTS public.help_time_weekly_2
#    ADD CONSTRAINT weekid PRIMARY KEY (weekid);

### Upload H_County_History 

In [86]:
# County
print(len(df_county))
dataTypeSeries = df_county.dtypes


58


In [83]:
# Only needed if the table is not empty
#cur.execute("DROP TABLE Help_County_2")

In [84]:
# the df_county can be uploaded to PostdegreSQL and a primary key can be added. 

cur.execute("CREATE TABLE IF NOT EXISTS Help_County_2 ( countyid INTEGER,  county_dought  varchar(255) ,county_fire varchar(255) )")
    

In [85]:
# upload the table in RDS (DWH)

# rename the DF - in this case the below function must not be changed
data = df_county

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  Help_County_2 (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()

In [ ]:
# add primary key in pg ADMIN: 

#ALTER TABLE Help_County 
#ADD PRIMARY KEY (countyid);

### Upload H_Help_Connect

In [87]:
# Connetion_ID
print(len(fire_key_df))
dataTypeSeries = fire_key_df.dtypes

1464


In [88]:
# upload the composite_key as a table - since joint key should alwayse have a uniquie mapping (the h_connection_key can be uploaded to PostdegreSQL and a primary key can be added)

cur.execute("CREATE TABLE IF NOT EXISTS help_connect_2 (  connectid INT, OBJECTID INT, Date date, Latitude float, Longitude FLOAT, Key varchar(255) )")

In [89]:
# Upload entries 

# rename the DF - in this case the below function must not be changed
data = fire_key_df

# Columns Definition 
cols = ",".join([str(i) for i in data.columns.tolist()])

#  uploaed each value to Postgree SQL 
for i,row in data.iterrows():
    sql = "INSERT INTO  help_connect_2   (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))

    # save all changes
    conn.commit()
    


In [ ]:
# add primary key in pg ADMIN: 
# ALTER TABLE h_connection_key ADD COLUMN keyid SERIAL PRIMARY KEY;

In [ ]:
#ALTER TABLE help_connect_2
#ADD PRIMARY KEY (connectid); 

In [ ]:
# Close the connection
cur.close()
conn.close()